# Extracción de nuevas características
Este script calcula nuevas métricas para cada sujeto a partir de los resultados de detección de FFT.
Lee el archivo CSV con los resultados de detección y calcula las siguientes métricas:
1. Tasa de detección
2. Varianza en el porcentaje de detección a través de los ensayos
3. Proporción de detección temprana (por debajo de 1.5s)
4. Asimetría de la potencia
5. Índice de robustez
6. Entropía de la detección a través de frecuencias
7. Respuestas fuertes (porcentaje > 66.6%)

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import entropy, skew
import os

# === Cargar datos ===
df = pd.read_csv("./results/fft_detection_results_combined.csv")
threshold = [1, 3, 5]

# Para cada umbral, se generará un nuevo archivo CSV con las métricas
for t in threshold:
    suffix = f"_{t}dB"

    # === Inicializar lista para almacenar las nuevas features por sujeto ===
    enhanced_metrics = []

    # === Agrupar por sujeto ===
    subjects = df["Subject"].unique()

    for subject in subjects:
        subject_data = df[df["Subject"] == subject]

        # Variables base
        signal_detected = subject_data["SignalDetected"].sum()
        avg_power = subject_data["AvgPower"].mean()
        power_variability = subject_data["PowerVariability"].mean()
        detected_time_pct = subject_data[f"DetectedTimePercentage{suffix}"].mean()
        detection_time = subject_data[f"DetectionTime{suffix}"].mean()

        # Feature 1: Detection Rate
        total_trials = subject_data["Trial"].nunique()
        detection_rate = signal_detected / (total_trials * subject_data["Freq"].nunique())

        # Feature 2: Variance in detection percentage across trials
        trial_detection_pct = subject_data.groupby("Trial")[f"DetectedTimePercentage{suffix}"].mean()
        detection_pct_var = trial_detection_pct.var()

        # Feature 3: Early detection ratio (below 1.5s)
        early_detection_ratio = (subject_data[f"DetectionTime{suffix}"] < 1.5).mean()

        # Feature 4: Skewness of power
        power_skew = skew(subject_data["AvgPower"])

        # Feature 5: Robustness Index
        robustness_index = detected_time_pct * (1 / detection_time) if detection_time > 0 else 0

        # Feature 6: Entropy of detection across frequencies
        freq_detection = subject_data.groupby("Freq")["SignalDetected"].mean()
        freq_entropy = entropy(freq_detection + 1e-9)  # small value to avoid log(0)

        # Feature 7: Strong responses (percentage > 66.6%)
        strong_responses = np.sum(subject_data[f"DetectedTimePercentage{suffix}"] > 66.6)

        enhanced_metrics.append({
            "Subject": subject,
            "SignalDetected": signal_detected,
            "AvgPower": avg_power,
            "PowerVariability": power_variability,
            "DetectedTimePercentage": detected_time_pct,
            "DetectionTime": detection_time,
            "DetectionRate": detection_rate,
            "DetectionPctVar": detection_pct_var,
            "EarlyDetectionRatio": early_detection_ratio,
            "PowerSkewness": power_skew,
            "RobustnessIndex": robustness_index,
            "FreqEntropy": freq_entropy,
            "StrongResponses": strong_responses
        })

    # Convertir a DataFrame y guardar
    enhanced_df = pd.DataFrame(enhanced_metrics)
    output_path = f"./results/enhanced_subject_metrics{t}.csv"
    enhanced_df.to_csv(output_path, index=False)
    output_path


# Clusterización de sujetos
Este script se encarga de cargar las métricas generadas por el script anterior y realizar la clasificación de los sujetos en función de su rendimiento en BCI e ITR.
Utiliza KNN como modelo de clasificación y guarda los resultados en un archivo CSV para cada umbral.

In [ ]:
import pandas as pd
import numpy as np
import itertools
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import os

# === Configuración ===
thresholds = [1, 3, 5]
metrics_template = "./results/enhanced_subject_metrics{thr}.csv"
rankings_file = "./results/subject_bci_itr.csv"
output_template = "./results/classification_results{thr}.csv"

# === Cargar etiquetas ===
def get_ranking_labels(path, metric):

    """
    Carga el archivo de rankings y asigna etiquetas a los sujetos
    según su posición en el ranking de la métrica especificada.
    """

    df = pd.read_csv(path)
    sorted_df = df.sort_values(by=metric, ascending=False).reset_index(drop=True)
    labels = {}
    n = len(df)
    for i, row in sorted_df.iterrows():
        if i < n / 3:
            labels[int(row["Subject"])] = 0  # Good
        elif i < 2 * n / 3:
            labels[int(row["Subject"])] = 2  # Mid
        else:
            labels[int(row["Subject"])] = 1  # Bad
    return labels

# === Ejecutar para cada threshold ===
for thr in thresholds:
    metrics_file = metrics_template.format(thr=thr)
    output_csv = output_template.format(thr=thr)

    # === Cargar métricas y etiquetas ===
    df = pd.read_csv(metrics_file)
    bci_labels = get_ranking_labels(rankings_file, "BCI")
    itr_labels = get_ranking_labels(rankings_file, "ITR")

    df["BCI_Class"] = df["Subject"].apply(lambda s: bci_labels.get(s, 2))
    df["ITR_Class"] = df["Subject"].apply(lambda s: itr_labels.get(s, 2))

    # === Definir métricas específicas del threshold actual ===
    feature_columns = [
        "SignalDetected", "AvgPower", "PowerVariability",
        "DetectedTimePercentage", "DetectionTime",
        "DetectionRate", "DetectionPctVar", "EarlyDetectionRatio",
        "PowerSkewness", "RobustnessIndex", "FreqEntropy", "StrongResponses"
    ]

    # === Ejecutar combinaciones ===
    all_results = []
    for r in range(1, 13):  # combinaciones de 2 a 12
        for combo in itertools.combinations(feature_columns, r):
            X = df[list(combo)]
            y_BCI = df["BCI_Class"]
            y_ITR = df["ITR_Class"]

            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)

            # === Dividir en conjuntos de entrenamiento y prueba ===
            X_train_bci, X_test_bci, y_train_bci, y_test_bci = train_test_split(X_scaled, y_BCI, test_size=0.3, random_state=42)
            X_train_itr, X_test_itr, y_train_itr, y_test_itr = train_test_split(X_scaled, y_ITR, test_size=0.3, random_state=42)

            for model_name, model in [
                ("KNN", KNeighborsClassifier(n_neighbors=3))
            ]:
                # === Para BCI ===
                model.fit(X_train_bci, y_train_bci)
                y_pred_bci = model.predict(X_test_bci)
                acc_bci = accuracy_score(y_test_bci, y_pred_bci)

                all_results.append({
                    "Threshold": f"{thr}dB",
                    "Target": "BCI",
                    "Model": model_name,
                    "Metrics": " + ".join(combo),
                    "Accuracy": acc_bci
                })

                # === Para ITR ===
                model.fit(X_train_itr, y_train_itr)
                y_pred_itr = model.predict(X_test_itr)
                acc_itr = accuracy_score(y_test_itr, y_pred_itr)

                all_results.append({
                    "Threshold": f"{thr}dB",
                    "Target": "ITR",
                    "Model": model_name,
                    "Metrics": " + ".join(combo),
                    "Accuracy": acc_itr
                })

    # === Guardar resultados ===
    results_df = pd.DataFrame(all_results)
    results_df.to_csv(output_csv, index=False)
    print(f"Resultados guardados en: {output_csv}")


Resultados guardados en: ./results/classification_results1.csv
Resultados guardados en: ./results/classification_results3.csv
Resultados guardados en: ./results/classification_results5.csv


# Visualización de resultados
Este script carga los resultados de clasificación y muestra las combinaciones más efectivas para cada umbral de detección.

In [ ]:
import pandas as pd

# Para cada umbral, cargar el CSV y mostrar los resultados
for thr in [1, 3, 5]:
    # Cargar CSV
    df = pd.read_csv(f"./results/classification_results{thr}.csv")
    print("\n" + "*" * 20)
    print(f"Resultados para {thr}dB:")

    # Ver las 10 combinaciones más efectivas para ITR
    top_itr = df[df["Target"] == "ITR"].sort_values(by="Accuracy", ascending=False).head(10)
    print("Top combinaciones para ITR:")
    print(top_itr[["Metrics", "Accuracy"] + [col for col in top_itr.columns if col.startswith("Imp_")]])

    # Ver las 10 combinaciones más efectivas para BCI
    top_bci = df[df["Target"] == "BCI"].sort_values(by="Accuracy", ascending=False).head(10)
    print("\nTop combinaciones para BCI:")
    print(top_bci[["Metrics", "Accuracy"] + [col for col in top_bci.columns if col.startswith("Imp_")]])


********************
Resultados para 1dB:
Top combinaciones para ITR:
                                                Metrics  Accuracy
121                     DetectionRate + RobustnessIndex  0.666667
41                     SignalDetected + RobustnessIndex  0.666667
6687  SignalDetected + AvgPower + PowerVariability +...  0.619048
1617  SignalDetected + AvgPower + PowerVariability +...  0.619048
33                       SignalDetected + DetectionRate  0.619048
3273  SignalDetected + AvgPower + PowerVariability +...  0.619048
881   SignalDetected + DetectionRate + RobustnessInd...  0.619048
1                                        SignalDetected  0.619048
5067  SignalDetected + AvgPower + PowerVariability +...  0.619048
11                                        DetectionRate  0.619048

Top combinaciones para BCI:
                                                Metrics  Accuracy
1398  DetectedTimePercentage + DetectionRate + Power...  0.809524
3308  SignalDetected + AvgPower + PowerVar

A partir de estos resultados podemos estudiar que combinaciones de características dan los mejores resultados de agrupamiento